<a href="https://colab.research.google.com/github/ponomarevaJV/PDA_SOL/blob/main/%D0%9B%D0%B0%D0%B1%D0%B0%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F%205(%D1%81%D0%BF%D0%B0%D1%80%D0%BA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
# Импорт модулей, не связанныех с PySpark.
import os
import sys
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import math
from IPython.core.interactiveshell import InteractiveShell
from datetime import *
import statistics as stats
# Это помогает автоматически распечатывать элементы без явного использования «печати».
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
! pip install pyspark

In [102]:
# Импорт модулей, связанных с PySpark.
import pyspark
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.functions import lit, desc, col, size, array_contains\
, isnan, udf, hour, array_min, array_max, countDistinct, count, sum, min, max
from pyspark.sql.types import *

MAX_MEMORY = '15G'
# Инициализировать сеанс Spark.
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY)
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Pyspark guide") \
        .config(conf=conf) \
        .getOrCreate()
    return spark

spark = init_spark()
filename_data = '/content/drive/MyDrive/2 семестр /Корпоративные платформы для хранения и анализа больших данных /5лб/vgsales.csv' # Загрузите данные в текущий каталог Colab.

df = spark.read.option("header", True).csv(filename_data)
print('Data frame type: ' + str(type(df)))

Data frame type: <class 'pyspark.sql.dataframe.DataFrame'>


In [103]:
print('Обзор данных')
df.printSchema()
print('Обзор столбцов')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Обзор данных
root
 |-- Rank: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: string (nullable = true)
 |-- EU_Sales: string (nullable = true)
 |-- JP_Sales: string (nullable = true)
 |-- Other_Sales: string (nullable = true)
 |-- Global_Sales: string (nullable = true)

Обзор столбцов


,Column Name,Data type
0,Rank,string
1,Name,string
2,Platform,string
3,Year,string
4,Genre,string
5,Publisher,string
6,NA_Sales,string
7,EU_Sales,string
8,JP_Sales,string
9,Other_Sales,string


In [104]:
print(f'Общее количество {df.count()} строк, печатаем несколько первых строк:')
df.limit(4).toPandas()

Общее количество 16598 строк, печатаем несколько первых строк:


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33


In [105]:
print('Обзор столбцов')
pd.DataFrame(df.dtypes, columns = ['Column Name','Data type'])

Обзор столбцов


,Column Name,Data type
0,Rank,string
1,Name,string
2,Platform,string
3,Year,string
4,Genre,string
5,Publisher,string
6,NA_Sales,string
7,EU_Sales,string
8,JP_Sales,string
9,Other_Sales,string


In [148]:
string_columns = ['Rank',	'Name',	'Platform',	'Year',	'Genre	Publisher']
numeric_columns = ['NA_Sales',	'EU_Sales',	'JP_Sales',	'Other_Sales',	'Global_Sales']
array_columns = []
missing_values = {}
for index, column in enumerate(df.columns):
    if column in string_columns:
 #       missing_count = df.filter(col(column).eqNullSafe(None) | col(column).isNull()).count()
        missing_values.update({column: missing_count})
    if column in numeric_columns:
        missing_count = df.where(col(column).isin([None, np.nan])).count()
        missing_values.update({column: missing_count})
    if column in array_columns:
        missing_count = df.filter(array_contains(df[column], 0) | array_contains(df[column], np.nan)).count()
        missing_values.update({column: missing_count})
missing_df = pd.DataFrame.from_dict([missing_values])
missing_df

,Rank,Name,Platform,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,0,0,0,0,0,0,0,0,0


In [151]:
for column in numeric_columns:
    df = df.withColumn(column, df[column].cast(FloatType()))

In [152]:
for column in string_columns:
    if missing_values['Platform'] > 0:
        df = df.withColumn(column, df(col(column).isNull(), "Platform").otherwise(col(column)))

In [153]:
for column in numeric_columns:
    if missing_values['Other_Sales'] > 0:
        df = df.withColumn(column, df(col(column).isNull() | col('Other_Sales').isin([np.nan]), 0).otherwise(col('Other_Sales')))

In [160]:
from pyspark.sql.functions import col, avg
grouped_df = df.groupBy("Name").agg(
    avg(col("NA_Sales")).alias("avg_NA_Sales"),
    avg(col("EU_Sales")).alias("avg_EU_Sales"),
    avg(col("JP_Sales")).alias("avg_JP_Sales"),
    avg(col("Other_Sales")).alias("avg_Other_Sales"),
    avg(col("Global_Sales")).alias("avg_Global_Sales")
)

In [161]:
grouped_df.show()


+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+
|                Name|        avg_NA_Sales|        avg_EU_Sales|       avg_JP_Sales|     avg_Other_Sales|   avg_Global_Sales|
+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+
|The Elder Scrolls...|  2.9100000460942588|   2.553333282470703|0.11666666716337204|  0.8466666539510092|  6.426666736602783|
|The Legend of Zel...|  0.9200000166893005|  0.5299999713897705| 0.4099999964237213| 0.05999999865889549| 1.9199999570846558|
|Call of Duty Blac...|   0.800000011920929|  0.5199999809265137|0.07000000029802322| 0.30000001192092896|  1.690000057220459|
|               Joust|  1.0099999904632568| 0.05999999865889549|                0.0|0.009999999776482582| 1.0800000429153442|
|Legacy of Kain: S...|  0.5799999833106995|  0.4000000059604645|                0.0| 0.07000000029802322| 1.0399999618

In [162]:
pandas_df = grouped_df.toPandas()

In [163]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,6))
plt.bar(pandas_df['Genre'], pandas_df['avg_Other_Sales'])
plt.title('Other Sales by Genre')
plt.xlabel('Genre')
plt.ylabel('Other Sales')
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
plt.show()